In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.preprocessing import StandardScaler, QuantileTransformer, OrdinalEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
from skopt import BayesSearchCV
import lightgbm as lgb
from lightgbm import LGBMRegressor
from joblib import dump, load
import pickle
import skops.io as sio
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml

In [ ]:
path_models = './models/'

In [ ]:
path = './ml_doc/'

In [ ]:
COUNT_ID = COUNT_ID_start = 147

In [ ]:
pred_results = []

# #146 run

RFE features

In [ ]:
csv = "combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_slctd_regions_6_OSMstart_metrics_OSMend.csv"
df = pd.read_csv(csv)
df = df[df['year'] != 2022]

df.drop(columns=['year', 'quarter', 'kindergartens_count_start', 'schools_count_start', 'sportsCentres_count_start', 'publicTransportStops_count_start', 'supermarkets_count_start', 'sum_of_count_lag2', 'sum_of_count_lag3', 'quantile_75_of_previous_month', 'kindergartens_count_end', 'schools_count_end', 'sportsCentres_count_end', 'publicTransportStops_count_end', 'supermarkets_count_end'], inplace=True)

df['month'] = df['month'].astype('category')
df['startClusterID'] = df['startClusterID'].astype('category')
df['endClusterID'] = df['endClusterID'].astype('category')
df['isSchoolHoliday'] = df['isSchoolHoliday'].astype('category')
df['isWeekend'] = df['isWeekend'].astype('category')
df['weekday_number'] = df['weekday_number'].astype('category')

train_ratio = 0.65
validation_ratio = 0.20
test_ratio = 0.15
train, test = train_test_split(df, test_size=1-train_ratio, random_state=42)
val, test = train_test_split(test, test_size=test_ratio/(test_ratio+validation_ratio), random_state=42)
X_train = train.drop('count_corrected', axis=1)
y_train = train['count_corrected']
X_val = val.drop('count_corrected', axis=1)
y_val = val['count_corrected']
X_test = test.drop('count_corrected', axis=1)
y_test = test['count_corrected']

# Define hyperparameter search space
param = {
    "regressor__max_depth": (1, 12),
    "regressor__learning_rate": (1e-3, 1),
    "regressor__min_split_gain":(0.0, 16384),
    "regressor__num_leaves": (48, 196),
    "regressor__min_child_weight": (0, 16384),
    "regressor__reg_alpha": (0.0, 16384),
    "regressor__reg_lambda": (0.0, 16384),
    "regressor__n_estimators": (640, 1640),
    "regressor__path_smooth": (0.0, 16384),
}

# Define categorical features
cat_feat = ["month", "startClusterID", "endClusterID",
            "isSchoolHoliday", "isWeekend", "weekday_number"]

# Create Pipeline with OrdinalEncoder
categorical_transformer = PMMLPipeline(
    steps=[
        ("encoder", OrdinalEncoder()),
    ]
)

# Set up Preprocessor with 'passthrough' of numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ("cat_feat", categorical_transformer, cat_feat)
    ], remainder="passthrough"
)

# Define Model Pipeline with Preprocessor and LightGBM
model = PMMLPipeline([
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(verbose=1,
                                force_row_wise=True,
                                callbacks=[lgb.early_stopping(5)],
                                eval_set=[(X_val, y_val)],
                                eval_metric="neg_mean_squared_error",
                                boosting_type="dart"))
])

# Configure Bayesian Search
search = BayesSearchCV(
    model,
    param,
    cv=5,
    scoring="neg_mean_squared_error",
    pre_dispatch=16,
    n_jobs=16,
    n_points=4,
    n_iter=196,
    error_score='raise'
)

# Fit the model to the training data
search.fit(X_train, y_train)


# Get the best model
best_model = search.best_estimator_
COUNT_ID = 147

# Evaluate the best model
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

pred_results.append({
    'count_id': COUNT_ID,
    'MSE': mse,
    'RMSE': rmse,
    'R2': r2
})

print("Mean Squared Error of Best Model:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared of Best Model:", r2)

unique_cluster_ids = df['startClusterID'].unique()
results_list = []
for cluster_id in unique_cluster_ids:
    # Filter data for the current cluster ID
    cluster_data = test[test['startClusterID'] == cluster_id]

    # calculate mean per cluster
    mean = cluster_data['count_corrected'].mean()

    # Extract features for prediction
    X_cluster = cluster_data.drop(['count_corrected'], axis=1)

    # Make predictions for the current cluster
    predictions = best_model.predict(X_cluster)

    # Calculate RMSE and R-squared
    y_true = cluster_data['count_corrected']
    rmse = np.sqrt(mean_squared_error(y_true, predictions))
    r2 = r2_score(y_true, predictions)

    # Get the count of samples
    count_samples = len(cluster_data)

    # Append results to the list
    results_list.append({'startClusterID': cluster_id,
                         'countOfSamples': count_samples,
                         'mean_count': mean,
                         'RMSE': rmse,
                         'R2': r2})

# Create a DataFrame from the list of results
results_df = pd.DataFrame(results_list)
results_df.to_csv(path + f'{COUNT_ID}' + '_pred_' + csv.split(sep='.')[0] + '.csv', index=False)

cv_results = pd.DataFrame(search.cv_results_)
fi = best_model.steps[1][1].feature_importances_
fi = [round(val, 3) for val in fi]
fi = pd.DataFrame(fi)
# Get the feature names from x_train
feature_names = X_train.columns.tolist()
# Create a new DataFrame with feature names and their corresponding importances
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': fi[0]})
cv_results.to_csv(path + f'{COUNT_ID}' + '_bs_' + csv.split(sep='.')[0] + '.csv', index=False)
feature_importances.to_csv(path + f'{COUNT_ID}' + '_fi_' + csv.split(sep='.')[0] + '.csv', index=False)

fn = path_models + f'{COUNT_ID}_' + csv.split(sep='.')[0]
with open(f'{fn}.pkl', 'wb') as file:
    pickle.dump(best_model, file)
dump(best_model, f'{fn}.joblib')
sio.dump(best_model, f'{fn}.skops')
sklearn2pmml(best_model, f"{fn}.pmml", with_repr = True)

pred_results_df = pd.DataFrame(pred_results)
pred_results_df.to_csv(path + f'{COUNT_ID_start}_{COUNT_ID}_prediction_results.csv', index=False)

In [ ]:
search


In [ ]:
pred_results_df = pd.DataFrame(pred_results)
pred_results_df.to_csv(path + f'{COUNT_ID_start}_{COUNT_ID}_prediction_results.csv', index=False)

In [ ]:
csv = "combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_slctd_regions_6_OSMstart_metrics_OSMend.csv"
df = pd.read_csv(csv)
df = df[df['year'] != 2022]
df.drop(columns=['year', 'quarter', 'kindergartens_count_start', 'schools_count_start', 'sportsCentres_count_start', 'publicTransportStops_count_start', 'supermarkets_count_start', 'sum_of_count_lag2', 'sum_of_count_lag3', 'quantile_75_of_previous_month', 'kindergartens_count_end', 'schools_count_end', 'sportsCentres_count_end', 'publicTransportStops_count_end', 'supermarkets_count_end'], inplace=True)
df['month'] = df['month'].astype('category')
df['startClusterID'] = df['startClusterID'].astype('category')
df['endClusterID'] = df['endClusterID'].astype('category')
df['isSchoolHoliday'] = df['isSchoolHoliday'].astype('category')
df['isWeekend'] = df['isWeekend'].astype('category')
df['weekday_number'] = df['weekday_number'].astype('category')

cat_feat = ["month", "startClusterID", "endClusterID", "isSchoolHoliday", "isWeekend", "weekday_number"]
categorical_transformer = PMMLPipeline(
    steps=[
        ("encoder", OrdinalEncoder()),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("cat_feat", categorical_transformer, cat_feat)
    ], remainder="passthrough"
)

train_ratio = 0.65
validation_ratio = 0.20
test_ratio = 0.15
train, test = train_test_split(df, test_size=1-train_ratio, random_state=42)
val, test = train_test_split(test, test_size=test_ratio/(test_ratio+validation_ratio), random_state=42)

test_encoded = preprocessor.fit_transform(test)
df_encoded = pd.DataFrame(test_encoded, columns=test.columns)

In [ ]:
df_encoded

In [ ]:
df_encoded.to_csv(csv.split('.')[0] + '__testEncoded.csv', index=False)

In [ ]:
csv = "combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_slctd_regions_6_OSMstart_metrics_OSMend.csv"
df = pd.read_csv(csv)
df = df[df['year'] != 2022]
df.drop(columns=['year', 'quarter', 'kindergartens_count_start', 'schools_count_start', 'sportsCentres_count_start', 'publicTransportStops_count_start', 'supermarkets_count_start', 'sum_of_count_lag2', 'sum_of_count_lag3', 'quantile_75_of_previous_month', 'kindergartens_count_end', 'schools_count_end', 'sportsCentres_count_end', 'publicTransportStops_count_end', 'supermarkets_count_end'], inplace=True)
df['month'] = df['month'].astype('category')
df['startClusterID'] = df['startClusterID'].astype('category')
df['endClusterID'] = df['endClusterID'].astype('category')
df['isSchoolHoliday'] = df['isSchoolHoliday'].astype('category')
df['isWeekend'] = df['isWeekend'].astype('category')
df['weekday_number'] = df['weekday_number'].astype('category')

train_ratio = 0.65
validation_ratio = 0.20
test_ratio = 0.15
train, test = train_test_split(df, test_size=1-train_ratio, random_state=42)
val, test = train_test_split(test, test_size=test_ratio/(test_ratio+validation_ratio), random_state=42)

df_test = pd.DataFrame(test, columns=test.columns)
df_test.to_csv(csv.split('.')[0] + '__test.csv', index=False)

In [ ]:
df_test

# read model

In [ ]:
import joblib
import pandas as pd

In [ ]:
# Load the saved model
model_filename = "./ml6/models/147_combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside.joblib"
loaded_model = joblib.load(model_filename)

In [ ]:
df_test = pd.read_csv('combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_slctd_regions_6_OSMstart_metrics_OSMend__test.csv')

In [ ]:
X_test = df_test.drop(columns='count_corrected')
y_test = df_test['count_corrected']


In [ ]:
# Evaluate the best model
y_pred = loaded_model.predict(X_test, verbose=-1)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error of Best Model:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared of Best Model:", r2)


In [ ]:
import math

In [ ]:
# Create an empty dictionary to store predicted and actual values
prediction_data = {'Predicted': [], 'Actual': [], 'Residual': [], 'RSS': []}

count = 1

# Loop through each row in X_test and predict
for index in range(len(X_test)):
    df_row = X_test.iloc[index:index+1]  # Get the current row as a DataFrame
    predicted_value = loaded_model.predict(df_row, verbose=-1)[0]  # Predict for the current row and get the first element
    actual_value = y_test.iloc[index]  # Get the corresponding actual value

    # Append the predicted and actual values to the dictionary
    prediction_data['Predicted'].append(predicted_value)
    prediction_data['Actual'].append(actual_value)
    prediction_data['Residual'].append(actual_value - predicted_value)
    prediction_data['RSS'].append(math.pow(actual_value - predicted_value, 2))

    if index == count:
        print(index)
        count += 5000

# Convert the prediction_data dictionary to a DataFrame
prediction_df = pd.DataFrame(prediction_data)

In [ ]:
# Convert the prediction_data dictionary to a DataFrame
prediction_df = pd.DataFrame(prediction_data)

In [ ]:
prediction_df

In [ ]:
prediction_df.RSS.mean()

In [ ]:
prediction_df.to_csv('combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_slctd_regions_6_OSMstart_metrics_OSMend__test_result.csv', index=False)

In [ ]:
pd.read_csv('combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_slctd_regions_6.csv')

In [ ]:
pd.read_csv("combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_slctd_regions_6_OSMstart_metrics_OSMend.csv")